In [1]:
# All imports needed
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures 

### Dataset Visualizations

Dataset used: [Real Estate dataset](http://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set)

##### Features:
* transcation date: the transaction date (for example, 2013.250=2013 March, 2013.500=2013 June, etc.)
* house age: the house age (unit: year)
* MRT distance: the distance to the nearest MRT station (unit: meter)
* number of stores: the number of convenience stores in the living circle on foot (integer)
* latitude: the geographic coordinate, latitude. (unit: degree)
* longitude: the geographic coordinate, longitude. (unit: degree)

The output is as follow
* price: house price of unit area (10000 New Taiwan Dollar/Ping, where Ping is a local unit, 1 Ping = 3.3 meter squared)

In [2]:
# Read data from file
real_estate = pd.read_csv("../data/metro/metro.csv") 
original_data = real_estate.copy()

#### Data exploration:

In [3]:
# original_data.info()
# original_data.describe()
# original_data.hist(figsize=(8,8))
# original_data.boxplot()
# original_data.shape

#### Preprocessing:
As the data set has a column of dates which is a float value, we decided it might increas the prediction quality if we separate according to the month and year. The value after the decimal point represents the month. Taking 100 divided by 12 (12 months) its values is the corresponding month if the valued is diveded by 100/12.
We are going to test regression with different preprocessed datasets:

* original_data: imported from the csv file, without any preprocessing
* real_estate: transaction date splitted into transaction year and month (additional columns)

To try out scaling, we decided (due to the fact that not all values are normally distributed) to try RobustScaling and MinMaxScaling.

In [4]:
# preprocessing date
# transaction_date = real_estate['transaction date'].astype(str).str.split('.', n = 1, expand = True)
# real_estate['transaction year'] = real_estate['transaction date'].apply(np.floor)
# real_estate['transaction month'] = (((abs(real_estate['transaction date']-real_estate['transaction year']))*1000)/83).round()
# real_estate = real_estate.drop(columns=['transaction date'])
# real_estate.head()

#### Modeling:

In [5]:
def polynomial_regression(X_train, X_test, y_train, degree = 2):
    poly = PolynomialFeatures(degree)
    x_poly = poly.fit_transform(X_train)
    
    model = LinearRegression()
    model.fit(x_poly, y_train)
    
    y_pred = model.predict(poly.fit_transform(X_test))
    return y_pred

def plot_scatter(y_test, y_pred):
    # Plot outputs
    plt.scatter(y_test, y_pred)

    plt.xticks(())
    plt.yticks(())

    plt.show()
    
def print_performance(y_test, y_pred):
    # Mean Squared Error
    print("MSE: ", metrics.mean_squared_error(y_test,y_pred))
    
    print("RMSE: ", metrics.mean_squared_error(y_test,y_pred, squared=False))
    
    # R2 is between 0 and 100 percent
    # 0 indicates that the model explains none of the variability of the response data around its mean.
    # 100 indicates that the model explains all the variability of the response data around its mean.
    print("R2: ", metrics.r2_score(y_test,y_pred) * 100)

In [6]:
# setting parameters for testing
# regression models
alpha = [0.1,0.9,1,2,4]
normalize = [True, False]
degrees = [2,3]

# knn
n_neighbour = [2, 5, 8, 12] 
weights = ['uniform', 'distance']

# trees
tree_max_depth = [6, 9, 12, 15]
tree_min_split = [3, 6, 9, 12]
forest_n_estimators = [10, 50, 75]
forest_max_depth = [6, 9, 12, 15, 20]
forest_min_split = [2, 5] 

#### Linear Models:

In [7]:
X_train, X_test, y_train, y_test = train_test_split(real_estate.drop('traffic_volume', axis=1),real_estate['traffic_volume'], test_size=0.2, random_state=44)

# preprocessing with scaler
# scaler = RobustScaler()
scaler = MinMaxScaler()
scaler_X_train = scaler.fit_transform(X_train)
scaler_X_test = scaler.transform(X_test)

# Linear Regression
print('Linear Regression')
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print_performance(y_test, y_pred)

# Lasso Regression
print('\nLasso Regression')
grid = GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': alpha,
                        'normalize':normalize})
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.predict(X_test)
print_performance(y_test, y_pred)

# Ridge Regression
print('\nRidge Regression')
grid = GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': alpha,
                        'normalize':normalize})
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.predict(X_test)
print_performance(y_test, y_pred)

# Polynomial Regression
print('\nPolynomial Regression Degree 2')
y_pred2 = polynomial_regression(X_train, X_test, y_train)
print_performance(y_test, y_pred2)
print('\nPolynomial Regression Degree 3')
y_pred3 = polynomial_regression(X_train, X_test, y_train, 3)
print_performance(y_test, y_pred3)

Linear Regression
MSE:  3319895.787575414
RMSE:  1822.0581186052805
R2:  15.880656681245597

Lasso Regression
{'alpha': 0.1, 'normalize': False}
MSE:  3320015.0595705356
RMSE:  1822.0908483307126
R2:  15.877634573761522

Ridge Regression
{'alpha': 2, 'normalize': False}
MSE:  3319868.602081635
RMSE:  1822.0506584839059
R2:  15.881345505844568

Polynomial Regression Degree 2
MSE:  1193017.5959715673
RMSE:  1092.253448596784
R2:  69.77138345232842

Polynomial Regression Degree 3
MSE:  1040487.744121782
RMSE:  1020.0430109175701
R2:  73.63617674557857


#### Non-parametric Model:

In [8]:
# knn regression
print('KNN')
grid = GridSearchCV(estimator=neighbors.KNeighborsRegressor(),
             param_grid={'n_neighbors': n_neighbour,
                        'weights': weights})
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.predict(X_test)
print_performance(y_test, y_pred)

KNN
{'n_neighbors': 8, 'weights': 'distance'}
MSE:  288368.27966347715
RMSE:  536.9993292951837
R2:  92.69333983011369


#### Tree Models:

In [9]:
# regression tree
print('Regression Tree')
grid = GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': tree_max_depth,
                        'min_samples_leaf': tree_min_split})
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.predict(X_test)
print_performance(y_test, y_pred)

# random forest
print('\nRandom Forest')
grid = GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'n_estimators': forest_n_estimators,
                         'max_depth': forest_max_depth,
                         'min_samples_leaf': forest_min_split})
grid.fit(X_train, y_train)
print(grid.best_params_)
y_pred = grid.predict(X_test)
print_performance(y_test, y_pred)

Regression Tree
{'max_depth': 12, 'min_samples_leaf': 12}
MSE:  245245.0382504528
RMSE:  495.2222109825576
R2:  93.78599423300655

Random Forest
{'max_depth': 15, 'min_samples_leaf': 2, 'n_estimators': 75}
MSE:  215914.07914906906
RMSE:  464.66555623272643
R2:  94.52918051847715
